<a href="https://colab.research.google.com/github/jean-kunz/cop_ml/blob/master/var_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# quantile regression

In ordinary linear regression, we are estimating the mean of some variable y, conditional on the values of independent variables X. 

As we proceed to fit the OLS regression model on the data we make a __key assumption about the random error term__ in the linear model. Our assumption is that the __error term has a constant variance__ across the values of independent variable X.

- What happens when this assumption is no longer true ?
- Also instead of estimating the mean of our independent variable can we estimate the median or the 0.3th quantile or 0.8th quantile of our independent variable ?.

This is where Quantile Regression comes to our rescue : A linear model to predict a given quantile of y, not the mean as in OLS 

__Quantile regression estimates the conditional median (or quantile) of the target__. Quantile regression is an extension of linear regression that is used when the conditions of linear regression are not met (i.e., linearity, homoscedasticity, independence, or normality).

For example (in a price prediction model), if we were to find the 25th quantile for the price of a particular home, that would mean that there is a 25% chance the actual price of the house is below the prediction, while there is a 75% chance that the price is above.

Now instead of being constants as in OLS, the regression coefficients ($\Theta$ or $\beta$) are now functions with a dependency on the quantile. For each quantile there are other coefs.

https://towardsdatascience.com/quantile-regression-ff2343c4a03

https://www.datasciencecentral.com/profiles/blogs/quantile-regression-in-python

### implementation
https://www.statsmodels.org/dev/examples/notebooks/generated/quantile_regression.html

https://colab.research.google.com/drive/1nXOlrmVHqCHiixqiMF6H8LSciz583_W2

In [ ]:
!pip install line_profiler

In [ ]:
%load_ext line_profiler

In [ ]:

#

import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt

data = sm.datasets.engel.load_pandas().data
data

In [ ]:
mod = smf.quantreg('foodexp ~ income', data)
res = mod.fit(q=.5)
print(res.summary())

In [ ]:
quantiles = np.arange(.05, .96, .1)
#quantiles = np.array([0.5])
def fit_model(q):
    #print(mod)    
    # fit with quantile regr model : mod
    res = mod.fit(q=q)
    return [q, res.params['Intercept'], res.params['income']] + \
            res.conf_int().loc['income'].tolist()

models = [fit_model(x) for x in quantiles]
models = pd.DataFrame(models, columns=['q', 'intercept', 'income', 'lower_ci_bound', 'upper_ci_bound'])

ols_model = smf.ols('foodexp ~ income', data).fit()
ols_ci = ols_model.conf_int().loc['income'].tolist()
ols = dict(intercept = ols_model.params['Intercept'],
           income = ols_model.params['income'],
           lower_ci_bound = ols_ci[0],
           upper_ci_bound = ols_ci[1])

print("quantile regr: \n",models)
print("ols: \n",ols)

In [ ]:
data.quantile(quantiles)

In [ ]:
models

In [ ]:
x = np.arange(data.income.min(), data.income.max(), 50)
get_y = lambda a, b: a + b * x

fig, ax = plt.subplots(figsize=(8, 6))

for i in range(models.shape[0]):
    y = get_y(models.intercept[i], models.income[i])
    #print("quantile:", quantiles[i])
    if 0.3<quantiles[i]<0.6:
        ax.plot(x, y, linestyle='dotted', color='black', label=quantiles[i])
    else:
        ax.plot(x, y, linestyle='dotted', color='lightblue')
    

y = get_y(ols['intercept'], ols['income'])

ax.plot(x, y, color='red', label='OLS')
ax.scatter(data.income, data.foodexp, alpha=.2)
ax.set_xlim((240, 3000))
ax.set_ylim((240, 2000))
legend = ax.legend()
ax.set_xlabel('Income', fontsize=16)
ax.set_ylabel('Food expenditure', fontsize=16);

In [ ]:
n = models.shape[0]
p1 = plt.plot(models.q, models.income, color='black', label='Quantile Reg.')
p2 = plt.plot(models.q, models.upper_ci_bound, linestyle='dotted', color='black')
p3 = plt.plot(models.q, models.lower_ci_bound, linestyle='dotted', color='black')
p4 = plt.plot(models.q, [ols['income']] * n, color='red', label='OLS')
p5 = plt.plot(models.q, [ols['lower_ci_bound']] * n, linestyle='dotted', color='red')
p6 = plt.plot(models.q, [ols['upper_ci_bound']] * n, linestyle='dotted', color='red')
plt.ylabel(r'$\beta_{income}$')
plt.xlabel('Quantiles of the conditional food expenditure distribution')
plt.legend()
plt.show()

## Value at risk prediction

In [ ]:
%load_ext tensorboard
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from absl import logging

import pandas as pd
from scipy.stats import norm
import scipy
import numpy as np
#%reload_ext google.colab.data_table
from IPython.display import display as dsp


In [ ]:
ls /content/drive/My\ Drive/Colab\ Notebooks/cop_ml/dataset/fh_5yrs.csv


Here is the paper we want to reproduce: https://arxiv.org/pdf/1908.07978v1.pdf

Data are located at kaggle: 
https://www.kaggle.com/qks1lver/amex-nyse-nasdaq-stock-histories?select=fh_5yrs.csv

We take the fh_5yrs.csv file that is uploaded to google drive. Then google drive is installed within collaboratory environment so one can access it.

In [ ]:
stocks_path = '/content/drive/My Drive/Colab Notebooks/cop_ml/dataset/fh_5yrs.csv'
stocks_df = pd.read_csv(stocks_path)
stocks_df.date = pd.to_datetime(stocks_df.date)
#%unload_ext google.colab.data_table
#stocks_df = stocks_df.loc[stocks_df.symbol.isin(['AAPL','TSLA'])]

In [ ]:
stocks_df['returns'] = stocks_df.sort_values(['symbol','date'], ascending=True).groupby(['symbol']).adjclose.pct_change()
fig = plt.figure()
fig.set_size_inches(10,3)
sns.lineplot(data=stocks_df.query("symbol=='AAPL'").sort_values('date', ascending=True), x="date", y="adjclose")
# first return of each symbol is nan -> set to 0
stocks_df.returns.fillna(0., inplace=True)


In [ ]:
fig = plt.figure()
fig.set_size_inches(10,3)

sns.lineplot(data=stocks_df.query('symbol=="AAPL"').sort_values('date', ascending=True), x='date', y='returns')

In [ ]:
stocks_df.query('symbol=="AAPL"').returns.hist(bins=40)

In [ ]:
Q = stocks_df.query('symbol=="AAPL"').returns.dropna()
scipy.stats.probplot(Q, dist=scipy.stats.norm, plot=plt.figure().add_subplot(111))
plt.title("Normal QQ-plot", weight="bold");

In [ ]:
tdf, tmean, tsigma = scipy.stats.t.fit(Q)
scipy.stats.probplot(Q, dist=scipy.stats.t, sparams=(tdf, tmean, tsigma), plot=plt.figure().add_subplot(111))
plt.title("Student QQ-plot", weight="bold");

### Normalization

https://developers.google.com/machine-learning/data-prep/transform/normalization

In [ ]:
# a few outliers
#plt.hist(stocks_df.adjclose, bins=20)
sns.histplot(data=stocks_df, x='adjclose', bins=20)


In [ ]:
stocks_df.loc[:,'log_adjclose']= np.log(stocks_df.adjclose)

In [ ]:
# log absorb outliers
sns.histplot(data=stocks_df, x='log_adjclose', bins=30)

In [ ]:
# normalize with z-score
stocks_df['z_adjclose']=np.divide((stocks_df.adjclose - stocks_df.adjclose.mean()),stocks_df.adjclose.std())
#stocks_df.adjclose.std

In [ ]:
sns.histplot(data=stocks_df, x='z_adjclose', bins=20)

log scaling looks like the one we should use


In [ ]:
from sklearn.preprocessing import power_transform
stocks_df.loc[:,'log_returns'] = power_transform(np.expand_dims(stocks_df.returns, axis=1),method='yeo-johnson')



In [ ]:
#sns.histplot(data=stocks_df, x='log_returns', bins=30)
sns.histplot(data=stocks_df, x='log_returns', bins=50)

## Returns vs Adjclose correlation

Check if they are correlated because both are candidate to be features

In [ ]:
stocks_df.query("symbol=='TSLA'")[['log_adjclose','returns']].corr()
stocks_df[['log_adjclose','returns']].corr()


# definition of VAR

If the 0.05 empirical quantile of daily returns is at 0.11. That means that with 95% confidence, our worst daily loss will not exceed 11%. If we have a 1 M€ investment, our one-day 5% VaR is 0.11 * 1 M€ = 34 k€.

Here we want to predict the maximum loss we can have with a confidence level of $1-\theta$, $\theta$ being the confidence level.

http://www.simonemanganelli.org/Simone/Research_files/caviarPublished.pdf

Traditional var assumes distribution of return doesn't change over time, at least during return measurement. 

It fit returns to a distribution (mean, std,...) and use quantile to define the thresehold perf. 

Here we try to predict VAR with a different method than historical var, which use distribution of daily return and returns the theta quantile of the worst daily pct returns

**We try to predict what will be the worst perf in the next 30 days**

In [ ]:
stocks_df.columns
stocks_df.symbol.unique()
symbol_list = stocks_df.symbol.unique()
np.random.shuffle(symbol_list)
symbol_list = symbol_list.tolist()[:100]
symbol_list.extend(['AAPL','TSLA'])


In [ ]:
df = stocks_df.loc[stocks_df.symbol.isin(symbol_list)]
#df = stocks_df

In [ ]:
df.date.max(), df.date.min(), stocks_df.date.max(), stocks_df.date.min()

In [ ]:
# compute the max loss (min neg returns) per stock on expanding window.
max_loss_df = df.sort_values(['symbol','date']).groupby(['symbol']) \
                    .expanding(1)[['returns']].min().reset_index()
max_loss_df.rename(columns={'returns':'max_loss'}, inplace=True)


In [ ]:
# to do a forward rolling window to find the next worst perf in 30 timesteps.
# we do sort by date in reverse order with a rolling window.  
max_loss_30_df = df.sort_values(['symbol','date'], ascending=False).groupby(['symbol']) \
                    .rolling(30, on='date', min_periods=1)[['returns']].min().reset_index()
max_loss_30_df.rename(columns={'returns':'max_loss_30'}, inplace=True)

In [ ]:
max_loss_30_df

In [ ]:
all_df = df.merge(max_loss_30_df, on=['symbol','date'], how='inner')
all_df.fillna(0.0, inplace=True)

In [ ]:
data_to_display = all_df.query('symbol=="AAPL"')[['date','returns','max_loss_30']]
sns.lineplot(x='date', y='value', hue='variable',
             data=pd.melt(data_to_display, ['date']))

In [ ]:
from datetime import date

bd = pd.tseries.offsets.BusinessDay(n = 30) 
date(2015,1,5) + bd


In [ ]:
all_df[(all_df.symbol=='AAPL')&(all_df.returns<0)].sort_values('date')[['date','returns','max_loss_30']].head(40)

In [ ]:
# implement a vectorized version

def rolling_win_padded(x, win_len):
    nb_row = x.shape[0]
    nb_col = x.shape[1]
    #print("nb rows in rolling padded", nb_row)

    # pad x with zeros left of win_len - 1 (there is at least one row in window on the right)
    pad = np.zeros((win_len-1, nb_col ))
    padded_x = np.concatenate([pad,x])
    #print("padded x", padded_x, "\n pad", pad)
    # create a vectorized index based on a rolling index
    # rolling index defines windows with index pointing to data in x
    win_dim = np.expand_dims(np.arange(win_len),0)
    timestep_dim = np.expand_dims(np.arange(nb_row),0).T
    rolling_idx = win_dim + timestep_dim
    #print("rolling idx", rolling_idx, "\n win dim", win_dim,"\n ts dim" , timestep_dim)
    return padded_x[rolling_idx]


def rolling_future_value(x, nb_future_steps=1):
    nb_row = x.shape[0]
    nb_col = x.shape[1]
    #print("nb rows in future value", nb_row)

    #print(np.expand_dims(np.arange(nb_future_steps),0))
    # standard case when there are more rows than future steps
    if nb_row >= nb_future_steps:
        base_idx = np.arange(nb_row-nb_future_steps)
        #print("base_idx",base_idx)
        future_idx = base_idx + nb_future_steps
        #print("future_idx", future_idx)    
        end_pad = np.zeros((nb_future_steps, nb_col))
        futur_x = x[future_idx]
        #print(x.shape, futur_x.shape, end_pad.shape)
        end_padded_x = np.concatenate([futur_x, end_pad])
    else:
        # return nb_row of zeros
        end_padded_x = np.zeros((nb_row, nb_col))
    return end_padded_x


x = np.array([[1,'a',1.5],
             [2, 'b', 3.2],
             [3, 'c', 3.5],
             [4, 'd', 3.3],
             [5, 'e', 5.2],
             [6, 'f', 8.2]])

rolling_x = rolling_win_padded(x[:3,:], 4)

#rolling_y = rolling_future_value(np.expand_dims(x[:,2],1), 2)
#rolling_y = rolling_future_value(x[:3,2:3], 5)

# if x.shape[0] < nb_fut_steps, it should return x.shape[0] 
rolling_y = rolling_future_value(x[:3,2:3], 1)

print("rolling x: \n",rolling_x,"\n rolling y\n" ,rolling_y)
assert rolling_x.shape[0]== rolling_y.shape[0], "should return same nb of rows"

In [ ]:
assert len(all_df[all_df.returns.isnull()==True])==0


In [ ]:
def rolling_win_vec(df, **kwargs):       
    input_cols = kwargs['input_cols']
    output_col = kwargs['output_col']   
    win_len = kwargs['win_len']
    sort_col_name = kwargs['sort_col_name']
    nb_pred_timesteps = kwargs['nb_pred_timesteps']
    #print(">> symbol:",df.symbol.unique()[0])
    x = np.array(df.sort_values(sort_col_name)[input_cols].values)
    y = np.array(df.sort_values(sort_col_name)[output_col].values)
    padded_win_x =  rolling_win_padded(x, win_len)    
    padded_y = rolling_future_value(y, nb_future_steps=nb_pred_timesteps)
    #print(len(df),y.shape, padded_win_x.shape, padded_y.shape)

    df['rolling_x']= padded_win_x.tolist()
    df['y']= padded_y.tolist()
    #print(">>", padded_win_x)
    return df

nb_pred_timesteps = 0 #we predict for the next 30 steps (open days) the max loss

def prep_features(df, symbols=None):
    if symbols is not None:
        grp_df = all_df.loc[all_df.symbol.isin(symbols)].sort_values(['symbol','date']).groupby('symbol')
    else:
        grp_df = all_df.sort_values(['symbol','date']).groupby('symbol')
    
    win_df = grp_df.apply(rolling_win_vec, input_cols=['log_adjclose','returns'],
                                              output_col=['max_loss_30'],
                                              win_len=128,
                                              nb_pred_timesteps = nb_pred_timesteps,
                                              sort_col_name='date')
    
    return win_df

symbol_list = all_df.symbol.unique()
np.random.shuffle(symbol_list)
symbols = [symbol_list[0]]
symbols = symbol_list[:]
stock_win_df = prep_features(df, symbols )


In [ ]:
stock_win_df.sort_values(['symbol','date'])[-60:-30][['symbol','date','rolling_x','max_loss_30','y']]

In [ ]:
X = np.array(stock_win_df.rolling_x.values.tolist())
y = np.array(stock_win_df.y.values.tolist())
y = np.reshape(y, (y.shape[0],1))
assert X.shape[0]== y.shape[0]

In [ ]:
def to_np(df):
    X = np.asarray(df.rolling_x.values.tolist()).astype('float64')    
    y = np.asarray(df.y.values.tolist()).astype('float64')
    y = np.reshape(y, (y.shape[0], 1))    
    return (X,y)

In [ ]:
train_df = stock_win_df.loc[stock_win_df.date < "2019-06-01"]
xval_df = stock_win_df.loc[(stock_win_df.date >='2019-06-01') & (stock_win_df.date < '2020-01-01')]
test_df = stock_win_df.loc[stock_win_df.date > "2020-01-01"][:]  # we omit last 5 var because they are nan (we predict 5 steps in advance)
train = to_np(train_df)
xval = to_np(xval_df)
test = to_np(test_df)

In [ ]:
train[0].shape, train[1].shape, xval[0].shape, xval[1].shape, test[0].shape, test[1].shape

# QCNN model

In [ ]:
import numpy as np
import tensorflow as tf
from datetime import datetime

In [ ]:
batch_size = 32
train_len = train[0].shape[0]

train_ds = tf.data.Dataset.from_tensor_slices(train)
xval_ds = tf.data.Dataset.from_tensor_slices(xval)
test_ds = tf.data.Dataset.from_tensor_slices(test)

train_ds = train_ds.shuffle(train_len).batch(batch_size)
xval_ds = xval_ds.batch(batch_size)

In [ ]:
def quantile_loss(q=0.5):

    def loss(y_true, y_pred):
        #print(q,y_true.shape, y_pred.shape)
        e = (y_true - y_pred)        
        return tf.keras.backend.mean(tf.keras.backend.maximum(q * e, (q - 1) * e),axis=-1)
   
    # Return a function
    return loss



In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', input_shape=train[0].shape[1:]),
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', dilation_rate=2),
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', dilation_rate=4),
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', dilation_rate=8),
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', dilation_rate=16),
    tf.keras.layers.Conv1D(8, 2, activation='relu', padding='causal', dilation_rate=32),
    tf.keras.layers.Conv1D(1, 1, activation='linear'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1)
])

optim = tf.keras.optimizers.Adam(learning_rate=0.001)

model.compile(optimizer=optim,
              loss= quantile_loss(0.05),
              metrics=['mae', quantile_loss(0.05)])
#                loss= tf.keras.losses.Huber() ,
#                metrics=['mae', tf.keras.losses.Huber()])



logdir = "logs/var_preds/qcnn/" + datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_cb = tf.keras.callbacks.TensorBoard(histogram_freq=2, log_dir=logdir)
early_stop_cb = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model.fit(train_ds,
         epochs=128,
         validation_data= xval_ds,
         callbacks=[tensorboard_cb, early_stop_cb])



In [ ]:
model.summary()

In [ ]:
model.evaluate(test[0], test[1])


In [ ]:
def prepare_stock_x(df, symbol, end_date, start_date):
    win_df = prep_features(df, [symbol] )    
    date_crit = (win_df.date<=end_date)&(win_df.date>=start_date)
    data = to_np(win_df[date_crit])    
    return data, win_df[date_crit].date.dt.date.values.tolist()
     

stock_period, dates = prepare_stock_x(df, 'FNDE', '2020-01-31','2019-01-01')
x = stock_period[0]
y = stock_period[1]
y_pred = model.predict(x)
preds = pd.DataFrame(list(zip(dates, y.squeeze(), y_pred.squeeze())), columns=['date','y','y_pred'])

sns.lineplot(x='date', y='value', hue='variable', data=pd.melt(preds, ['date']))

In [ ]:
df.symbol.unique() 

In [ ]:
%tensorboard --logdir logs/var_preds/qcnn
